# Consulta de titulares de los periódicos
Mediante las librerías Requests y BeautifulSoup se hace webscraping a los periódicos más destacados en España para mostrar el titular principal con el que abren sus portadas. El resultado, además de imprimirlo por pantalla, lo grabamos en un archivo.

**NOTA IMPORTANTE**. Muchos diarios tienen en su aviso legal o en sus robots.txt la prohibición de hacer web scraping a sus contenidos. Con este pequeño programa solo se muestra, a efectos didácticos, cómo usar los comandos y funciones de Python en este sentido.

In [2]:
import requests # Para hacer la consulta
from IPython.display import display, HTML # Para mostrar html en celdas de Jupiter Labs
from datetime import datetime, timedelta # Para menejar fechas en Python
from babel.dates import format_date, format_time # Para convertir las fechas y horas a español
from bs4 import BeautifulSoup #Para tratar el html de cada periódico
import re # Para manejar expresiones regulares

In [3]:
periodicos = {
    'El País': 'https://elpais.com/', 
    'El Mundo':'https://www.elmundo.es/', 
    'El Español':'https://www.elespanol.com/', 
    'El Periódico de España':'https://www.epe.es/es/',
    'ABC':'https://www.abc.es/', 
    'La Vanguardia':'https://www.lavanguardia.com/', 
    'El Periódico':'https://www.elperiodico.com/es/', 
    'El Confidencial':'https://www.elconfidencial.com/', 
    '20 Minutos':'https://www.20minutos.es/', 
    'La Razón':'https://www.larazon.es/', 
    'elDiario.es':'https://www.eldiario.es/', 
    'Público':'https://www.publico.es/', 
    'Infolibre':'https://www.infolibre.es/', 
    'El Independiente':'https://www.elindependiente.com/', 
    'El Correo de Andalucía':'https://www.elcorreoweb.es', 
    'Diario de Sevilla':'https://www.diariodesevilla.es/', 
    'ABC de Sevilla':'https://www.abc.es/sevilla/'
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

### Consultar primero qué etiquetas o clases tienen los titulares
Hay que entrar en cada web y ver con qué etiqueta (h1, h2,...) y dentro como se configura en el html. Por ejemplo, si es el primer h1 de la página (o h2), si tiene etiquetas adicionales dentro, clases, etc. para hacer la búsqueda y el muestreo correcto del titular.

**NOTA:** Hay que tener igualmente en cuenta que las distintas webs de los periódicos pueden cambiar el html, y si ahora el titular de apertura es un H1, en siguientes revisiones de diseño puede ser un H2.

**Función consultar_titular(a,b)**. Esta función recibe dos argumentos string: el nombre del periódico y la URL. El resultado es que muestra el nombre del periódico y el titular principal. Aquí hacemos el request y con la respuesta y BeautifulSoup encontramos el titular. No retorna nada, solo hace toda la gestión y muestra el titular.

In [6]:
def consultar_titular(a,b,contenido_HTML):
    try:
        response = requests.get(b,headers=headers)
        soup = BeautifulSoup(response.text,'html5lib')
        if a == "El Mundo":
            titular = soup.find_all('h2',class_='ue-c-cover-content__headline')[0].text
            enlace = soup.find_all('h2',class_='ue-c-cover-content__headline')[0].parent.get('href')
        elif a == "El Español" or a == "Diario de Sevilla":
            titular = soup.find_all('h2')[0].text.strip()
            if a == "El Español":
                enlace = 'https://www.elespanol.com' + soup.find_all('h2')[0].a.get('href')
            else:
                enlace = soup.find_all('h2')[0].parent.get('href')
        elif a == "El Correo de Andalucía" or a == "El Confidencial":
            titular = soup.find_all('h1')[1].text.strip()
            enlace = soup.find_all('h1')[1].parent.get('href')
        elif a == "20 Minutos":
            titular = soup.find_all('h1', id='')[0].a.text.strip()
            enlace = soup.find_all('h1', id='')[0].a.get('href')
        elif a == "La Razón":
            titular = soup.find_all('h2',class_='article__title')[0].a.text.strip()
            enlace = soup.find_all('h2',class_='article__title')[0].a.get('href')
        elif a == "Infolibre":
            titular = soup.find_all('h1')[0].text.strip()
            enlace = 'https://www.infolibre.es' + soup.find_all('h1')[0].a.get('href')
        else:
            titular = soup.find_all('h2')[0].a.text.strip()
            if a == "El Periódico de España":
                enlace = 'https://www.epe.es' + soup.find_all('h2')[0].a.get('href')
            elif a == "La Vanguardia": 
                enlace = 'https://www.lavanguardia.com' + soup.find_all('h2')[0].a.get('href')
            elif a == "El Periódico":
                enlace = 'https://www.elperiodico.com' + soup.find_all('h2')[0].a.get('href')
            elif a == "elDiario.es":
                enlace = 'https://www.eldiario.es' + soup.find_all('h2')[0].a.get('href')
            elif a == "Público":
                enlace = 'https://www.publico.es' + soup.find_all('h2')[0].a.get('href')
            else:
                enlace = soup.find_all('h2')[0].a.get('href')
                
        display(HTML(f'<h2>{a}</h2>'))
        contenido_HTML += f'<h2>{a}</h2>\n'
        display(HTML(f'<h3 style="line-height:1.4">{titular}</h3>'))
        contenido_HTML += f'<h3 style="line-height:1.4">{titular}</h3>\n'
        display(HTML(f'<a href="{enlace}" target="_blank">{enlace}</a>'))
        contenido_HTML += f'<a href="{enlace}" target="_blank">{enlace}</a>\n'
        display(HTML(f'<hr>'))
        contenido_HTML += f'<hr>\n'
    except requests.exceptions.RequestException as e:
        print(f"Error con {a}: {e}")
    except Exception as e:
        print(f"Error inesperado en {a}: {e}")
    return contenido_HTML

### Aquí defino las dos funciones para formatear las fechas de hoy y la hora actual de consulta

In [8]:
# Función que devuelve la fecha de hoy en español
# Ejemplo: lunes 3 de febrero de 2025
def hoy():
    hoy = datetime.now()
    fecha_español = format_date(hoy, format="full", locale="es")
    return fecha_español

# Función que devuelve la hora en este momento en español
# Ejemplo: 14:35
def ahora():
    ahora = datetime.now()
    hora_espanol = format_time(ahora, format='short', locale='es_ES')
    return hora_espanol

**Bloque principal**. Mediante un bucle se va recorriendo el diccionario de periódicos y llamando a la función correspondiente para que haga el request y muestre el titular.

**contenido_HTML**. En esta variable se va a ir almacenando lo que se va imprimiendo en pantalla, para después grabarlo en el fichero 'titulares-periodico.htm'.

**titulares_periodico.htm**. Aquí se irá guardando todo el contenido html, desde el DOCTYPE hasta el cierre de HTML, y dentro los titulares.

In [10]:
contenido_HTML = f"<h1>Titulares del {hoy()}</h1>\n<h3>Hora de la consulta: {ahora()}</h3>\n<hr>\n"
display(HTML(f'<h1>Titulares del {hoy()}</h1>'))
display(HTML(f'<h4>Hora de la consulta: {ahora()}</h4>'))
display(HTML(f'<hr>'))
for a,b in periodicos.items():
    contenido_HTML = consultar_titular(a,b,contenido_HTML)

with open('titulares_periodico.htm', 'w', encoding='utf-8') as file:
    file.write(f"<!DOCTYPE html>\n<html lang='es'>\n<head>\n<meta charset='utf-8'>\n<meta name='viewport' content='width=device-width, initial-scale=1.0'>\n<title>Titulares de periódicos</title>\n</head>\n<body>\n{contenido_HTML}\n</body>\n</html>")